<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/217_Cross_Sell_Upsell_Orchestrator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cross-Sell & Upsell Orchestrator - Phase 0 Planning Document

**Created:** Based on orchestrator agent development guides and project requirements

---

## 📊 Data Structure Analysis

### Customer Data Structure (`customers.json`)
- **Format:** JSON array of customer objects
- **Key Fields:**
  - `customer_id`: Unique identifier (e.g., "C001")
  - `name`, `email`: Identity fields
  - `loyalty_tier`: "gold", "silver", "bronze"
  - `churn_risk`: Float 0.0-1.0
  - `products_owned`: Array of `{product_id, purchase_date, amount}`
  - `purchase_history`: Array of `{product_id, date, amount}` (duplicate of products_owned?)
  - `categories`: Array of category strings (derived from products_owned)
  - `rfm`: `{recency_days, frequency_90d, monetary_90d}`
  - `lifetime_value`: Float
  - `price_sensitivity`: "low", "medium", "high"
  - `notes`: Human-readable notes (contains routine gaps mentioned)

**Observations:**
- `products_owned` and `purchase_history` appear to be duplicates
- `categories` is derived from products (not always present in all customers)
- `notes` field contains valuable business context (routine gaps)
- RFM metrics are pre-calculated
- Churn risk is pre-calculated

### Product Catalog Structure (`product_catalog.json`)
- **Format:** JSON array of product objects
- **Key Fields:**
  - `product_id`: Unique identifier (e.g., "P001")
  - `name`: Product name
  - `category`: Single category string (e.g., "cleanser", "toner", "serum")
  - `price`: Float
  - `margin`: "low", "medium", "high"
  - `replenishment_cycle_days`: Integer (days until replenishment needed)
  - `recommended_cross_sells`: Array of product_ids

**Observations:**
- Each product has one category (not multiple)
- Cross-sell relationships are pre-defined in `recommended_cross_sells`
- Replenishment cycles vary (20-45 days)
- Margin information available for prioritization

---

## 🎯 Business Rules & Decision Logic

### Core Routine Categories (Tier 1 Essentials)
Based on product catalog, essential categories:
1. **cleanser** - Foundation step
2. **toner** - Second step
3. **serum** - Treatment step
4. **moisturizer** - Hydration step
5. **spf** - Protection step
6. **mask** - Optional treatment
7. **lip** - Optional care
8. **mist** - Optional hydration
9. **eye** - Optional specialized care

**Essential Routine:** cleanser, toner, serum, moisturizer, spf (5 core products)

### Cross-Sell Logic
1. **Routine Gap Detection:**
   - Identify missing essential categories
   - Prioritize: cleanser → toner → serum → moisturizer → spf
   - Consider customer's existing products for compatibility

2. **Cross-Sell from Product Relationships:**
   - Use `recommended_cross_sells` from product catalog
   - If customer owns Product A, recommend products in its `recommended_cross_sells` list
   - Filter out products customer already owns

3. **Category-Based Recommendations:**
   - If customer has no product in essential category, recommend products from that category
   - Prioritize by: margin, price (based on price_sensitivity), replenishment cycle

### Upsell Logic
1. **Replenishment Upsell:**
   - If customer's product purchase_date + replenishment_cycle_days < today, suggest replenishment
   - Calculate days_since_purchase from purchase_date
   - Flag products approaching or past replenishment date

2. **Bundle Upsell:**
   - If customer has 2+ products, suggest complementary products to complete routine
   - Higher-value customers (high CLV, low price_sensitivity) → suggest premium bundles

3. **Category Upgrade:**
   - If customer has basic product, suggest premium alternative (future: when we have tier data)

### Opportunity Scoring Logic
Score each opportunity by:
1. **Business Value (40%):**
   - Product price × margin_multiplier
   - Margin multipliers: "high" = 1.5, "medium" = 1.0, "low" = 0.7

2. **Customer Fit (30%):**
   - Price sensitivity match: low sensitivity → higher prices OK, high sensitivity → lower prices
   - Loyalty tier: gold = 1.2, silver = 1.0, bronze = 0.8
   - Churn risk: high churn → higher urgency score

3. **Routine Completeness (20%):**
   - Essential category (cleanser, toner, serum, moisturizer, spf) = 1.5x
   - Optional category = 1.0x
   - Missing essential = 1.3x urgency

4. **Replenishment Urgency (10%):**
   - Products past replenishment date = 1.5x
   - Products approaching (within 7 days) = 1.2x
   - New products = 1.0x

**Final Score = (Business Value × 0.4) + (Customer Fit × 0.3) + (Routine Completeness × 0.2) + (Replenishment Urgency × 0.1)**

---

## 📐 State Schema Design

```python
class CrossSellUpsellState(TypedDict, total=False):
    # Input
    customer_id: str                        # Customer to analyze
    
    # Data Ingestion
    customer_data: Dict[str, Any]          # Loaded customer record
    product_catalog: List[Dict[str, Any]]  # All products
    product_lookup: Dict[str, Dict]        # product_id -> product dict (for fast lookup)
    
    # Routine Analysis
    customer_products: List[str]            # List of product_ids customer owns
    customer_categories: List[str]          # Categories customer has products in
    routine_gaps: List[str]                 # Missing essential categories
    replenishment_needs: List[Dict]        # Products needing replenishment
    # Structure: [{"product_id": "P001", "days_since_purchase": 45, "replenishment_due": True}]
    
    # Opportunity Detection
    cross_sell_opportunities: List[Dict]    # Cross-sell opportunities
    upsell_opportunities: List[Dict]        # Upsell opportunities
    # Structure per opportunity:
    # {
    #   "product_id": "P002",
    #   "product_name": "Balancing Facial Toner",
    #   "category": "toner",
    #   "price": 12.99,
    #   "margin": "medium",
    #   "recommendation_type": "routine_gap" | "product_cross_sell" | "replenishment",
    #   "rationale": "Customer missing essential toner step",
    #   "raw_score": float,
    #   "business_value_score": float,
    #   "customer_fit_score": float,
    #   "routine_completeness_score": float,
    #   "replenishment_urgency_score": float
    # }
    
    # Scoring & Ranking
    scored_opportunities: List[Dict]        # All opportunities with final scores
    ranked_opportunities: List[Dict]        # Sorted by final_score (descending)
    top_opportunities: List[Dict]           # Top N opportunities (configurable)
    
    # Summary Metrics
    opportunity_summary: Dict[str, Any]
    # Structure:
    # {
    #   "total_cross_sell_opportunities": int,
    #   "total_upsell_opportunities": int,
    #   "total_potential_revenue": float,
    #   "routine_completeness_percent": float,
    #   "replenishment_urgency_count": int,
    #   "high_value_opportunities": int  # Opportunities with score > threshold
    # }
    
    # Output
    recommendations_report: str            # Markdown report
    report_file_path: Optional[str]       # Path to saved report
    
    # Metadata
    errors: List[str]                      # Any errors encountered
    processing_time: Optional[float]       # Time taken to process
```

---

## 🏗️ Architecture Plan

### Node Structure (Linear Workflow)
1. **data_ingestion_node** - Load customer and product data
2. **routine_analysis_node** - Analyze customer routine, identify gaps, check replenishment
3. **opportunity_detection_node** - Find cross-sell and upsell opportunities
4. **scoring_ranking_node** - Score and rank opportunities
5. **report_generation_node** - Generate markdown report

### Utility Structure
**Data Loading Utilities:**
- `load_customer_data(customer_id: str) -> Dict`
- `load_product_catalog() -> List[Dict]`
- `build_product_lookup(catalog: List[Dict]) -> Dict[str, Dict]`

**Business Logic Utilities:**
- `identify_routine_gaps(customer_categories: List[str], essential_categories: List[str]) -> List[str]`
- `check_replenishment_needs(customer_products: List[Dict], product_catalog: List[Dict]) -> List[Dict]`
- `find_cross_sell_opportunities(customer_data: Dict, product_catalog: List[Dict], product_lookup: Dict) -> List[Dict]`
- `find_upsell_opportunities(customer_data: Dict, product_catalog: List[Dict], replenishment_needs: List[Dict]) -> List[Dict]`
- `score_opportunity(opportunity: Dict, customer_data: Dict, product: Dict) -> Dict`
- `rank_opportunities(opportunities: List[Dict]) -> List[Dict]`
- `calculate_opportunity_summary(opportunities: List[Dict]) -> Dict`

**Report Generation Utilities:**
- `generate_recommendations_report(state: CrossSellUpsellState) -> str`

---

## 🧪 Testing Strategy

### Phase 1: Test Utilities Independently
1. Test `load_customer_data("C001")` - verify structure
2. Test `load_product_catalog()` - verify all products loaded
3. Test `identify_routine_gaps()` - verify gap detection
4. Test `check_replenishment_needs()` - verify replenishment logic
5. Test `find_cross_sell_opportunities()` - verify opportunity detection
6. Test `score_opportunity()` - verify scoring logic

### Phase 2: Test Nodes Incrementally
1. Test `data_ingestion_node` - verify data loaded correctly
2. Test `routine_analysis_node` - verify gaps and replenishment detected
3. Test `opportunity_detection_node` - verify opportunities found
4. Test `scoring_ranking_node` - verify scoring and ranking
5. Test `report_generation_node` - verify report format

### Phase 3: Integration Testing
1. Test full workflow with customer C001 (has gaps: toner, serum, SPF)
2. Test with customer C002 (minimal routine, high churn risk)
3. Test with customer C004 (strong routine, missing moisturizer/SPF)

---

## 📝 Implementation Order

1. ✅ **Phase 0: Planning** (this document)
2. **Phase 1: Data Utilities** - Build and test data loading utilities
3. **Phase 2: Business Logic Utilities** - Build and test business logic
4. **Phase 3: State Schema** - Add to config.py
5. **Phase 4: State Inspection Tools** - Build debugging tools
6. **Phase 5: Nodes** - Build nodes incrementally, test each
7. **Phase 6: Workflow Integration** - Wire into LangGraph
8. **Phase 7: Testing & Refinement** - End-to-end testing

---

## 🎯 Success Criteria

- [ ] Can load customer and product data correctly
- [ ] Can identify routine gaps accurately
- [ ] Can detect replenishment needs
- [ ] Can find cross-sell opportunities
- [ ] Can find upsell opportunities
- [ ] Can score opportunities by business value
- [ ] Can rank opportunities correctly
- [ ] Can generate readable markdown report
- [ ] Full workflow runs end-to-end without errors
- [ ] Recommendations make business sense

---

*This plan follows the orchestrator agent development patterns from ORCHESTRATOR_AGENTS_GUIDE_2.md*



# fix data issues

In [ ]:
[
  {
    "customer_id": "C001",
    "name": "Sarah Lee",
    "email": "sarah.lee@example.com",
    "loyalty_tier": "gold",
    "churn_risk": 0.12,
    "products_owned": [
      {
        "product_id": "P001",
        "purchase_date": "2024-01-10",
        "amount": 14.99
      },
      {
        "product_id": "P004",
        "purchase_date": "2024-02-03",
        "amount": 17.99
      }
    ],
    "purchase_history": [
      {
        "product_id": "P001",
        "date": "2024-01-10",
        "amount": 14.99
      },
      {
        "product_id": "P004",
        "date": "2024-02-03",
        "amount": 17.99
      }
    ],
    "categories": [
      "cleanser",
      "moisturizer"
    ],
    "rfm": {
      "recency_days": 12,
      "frequency_90d": 2,
      "monetary_90d": 32.98
    },
    "lifetime_value": 210.40,
    "price_sensitivity": "medium",
    "notes": "High-value repeat customer; routine missing toner, serum, and SPF."
  },
  {
    "customer_id": "C002",
    "name": "Mark Johnson",
    "email": "mark.j@example.com",
    "loyalty_tier": "silver",
    "churn_risk": 0.28,
    "products_owned": [
      {
        "product_id": "P002",
        "purchase_date": "2024-01-05",
        "amount": 12.99
      }
    ],
    "purchase_history": [
      {
        "product_id": "P002",
        "date": "2024-01-05",
        "amount": 12.99
      }
    ],
    "categories": [
      "toner"
    ],
    "rfm": {
      "recency_days": 47,
      "frequency_90d": 1,
      "monetary_90d": 12.99
    },
    "lifetime_value": 89.50,
    "price_sensitivity": "high",
    "notes": "Recently reported delivery issues; extremely incomplete routine."
  },
  {
    "customer_id": "C003",
    "name": "Emily Chen",
    "email": "emily.chen@example.com",
    "loyalty_tier": "bronze",
    "churn_risk": 0.08,
    "products_owned": [
      {
        "product_id": "P003",
        "purchase_date": "2024-02-11",
        "amount": 19.99
      },
      {
        "product_id": "P007",
        "purchase_date": "2024-02-15",
        "amount": 6.99
      }
    ],
    "purchase_history": [
      {
        "product_id": "P003",
        "date": "2024-02-11",
        "amount": 19.99
      },
      {
        "product_id": "P007",
        "date": "2024-02-15",
        "amount": 6.99
      }
    ],
    "categories": [
      "serum",
      "lip"
    ],
    "rfm": {
      "recency_days": 10,
      "frequency_90d": 2,
      "monetary_90d": 26.98
    },
    "lifetime_value": 142.10,
    "price_sensitivity": "medium",
    "notes": "New customer who prefers hydrating products; routine missing cleanser, moisturizer, SPF."
  },
  {
    "customer_id": "C004",
    "name": "David Brooks",
    "email": "david.b@example.com",
    "loyalty_tier": "gold",
    "churn_risk": 0.35,
    "products_owned": [
      {
        "product_id": "P001",
        "purchase_date": "2023-12-20",
        "amount": 14.99
      },
      {
        "product_id": "P002",
        "purchase_date": "2023-12-20",
        "amount": 12.99
      },
      {
        "product_id": "P003",
        "purchase_date": "2024-01-02",
        "amount": 19.99
      }
    ],
    "purchase_history": [
      {
        "product_id": "P001",
        "date": "2023-12-20",
        "amount": 14.99
      },
      {
        "product_id": "P002",
        "date": "2023-12-20",
        "amount": 12.99
      },
      {
        "product_id": "P003",
        "date": "2024-01-02",
        "amount": 19.99
      }
    ],
    "categories": [
      "cleanser",
      "toner",
      "serum"
    ],
    "rfm": {
      "recency_days": 38,
      "frequency_90d": 3,
      "monetary_90d": 47.97
    },
    "lifetime_value": 310.75,
    "price_sensitivity": "low",
    "notes": "High-value but current churn risk due to unresolved ticket; strong routine builder missing moisturizer and SPF."
  },
  {
    "customer_id": "C005",
    "name": "Alicia Gomez",
    "email": "alicia.g@example.com",
    "loyalty_tier": "silver",
    "churn_risk": 0.15,
    "products_owned": [
      {
        "product_id": "P004",
        "purchase_date": "2024-02-05",
        "amount": 17.99
      },
      {
        "product_id": "P009",
        "purchase_date": "2024-01-26",
        "amount": 18.99
      }
    ],
    "purchase_history": [
      {
        "product_id": "P004",
        "date": "2024-02-05",
        "amount": 17.99
      },
      {
        "product_id": "P009",
        "date": "2024-01-26",
        "amount": 18.99
      }
    ],
    "categories": [
      "moisturizer",
      "eye"
    ],
    "rfm": {
      "recency_days": 20,
      "frequency_90d": 2,
      "monetary_90d": 36.98
    },
    "lifetime_value": 175.20,
    "price_sensitivity": "medium",
    "notes": "Routine focused on hydration and under-eye care; missing cleanser, toner, serum, SPF."
  },
  {
    "customer_id": "C006",
    "name": "Jason Patel",
    "email": "jason.p@example.com",
    "loyalty_tier": "bronze",
    "churn_risk": 0.10,
    "products_owned": [
      {
        "product_id": "P005",
        "purchase_date": "2024-02-10",
        "amount": 15.99
      }
    ],
    "purchase_history": [
      {
        "product_id": "P005",
        "date": "2024-02-10",
        "amount": 15.99
      }
    ],
    "categories": [
      "spf"
    ],
    "rfm": {
      "recency_days": 11,
      "frequency_90d": 1,
      "monetary_90d": 15.99
    },
    "lifetime_value": 95.30,
    "price_sensitivity": "medium",
    "notes": "New customer with only SPF; extremely incomplete routine missing cleanser, toner, serum, moisturizer."
  },
  {
    "customer_id": "C007",
    "name": "Rachel Kim",
    "email": "rachel.kim@example.com",
    "loyalty_tier": "gold",
    "churn_risk": 0.05,
    "products_owned": [
      {
        "product_id": "P001",
        "purchase_date": "2024-01-15",
        "amount": 14.99
      },
      {
        "product_id": "P002",
        "purchase_date": "2024-01-15",
        "amount": 12.99
      },
      {
        "product_id": "P003",
        "purchase_date": "2024-01-20",
        "amount": 19.99
      },
      {
        "product_id": "P004",
        "purchase_date": "2024-01-20",
        "amount": 17.99
      },
      {
        "product_id": "P005",
        "purchase_date": "2024-01-25",
        "amount": 15.99
      }
    ],
    "purchase_history": [
      {
        "product_id": "P001",
        "date": "2024-01-15",
        "amount": 14.99
      },
      {
        "product_id": "P002",
        "date": "2024-01-15",
        "amount": 12.99
      },
      {
        "product_id": "P003",
        "date": "2024-01-20",
        "amount": 19.99
      },
      {
        "product_id": "P004",
        "date": "2024-01-20",
        "amount": 17.99
      },
      {
        "product_id": "P005",
        "date": "2024-01-25",
        "amount": 15.99
      }
    ],
    "categories": [
      "cleanser",
      "toner",
      "serum",
      "moisturizer",
      "spf"
    ],
    "rfm": {
      "recency_days": 18,
      "frequency_90d": 5,
      "monetary_90d": 81.95
    },
    "lifetime_value": 450.60,
    "price_sensitivity": "low",
    "notes": "Complete routine customer; ideal for upsell to premium products or additional treatments (mask, eye cream)."
  },
  {
    "customer_id": "C008",
    "name": "Michael Torres",
    "email": "michael.t@example.com",
    "loyalty_tier": "silver",
    "churn_risk": 0.22,
    "products_owned": [
      {
        "product_id": "P001",
        "purchase_date": "2023-11-10",
        "amount": 14.99
      },
      {
        "product_id": "P004",
        "purchase_date": "2023-11-10",
        "amount": 17.99
      }
    ],
    "purchase_history": [
      {
        "product_id": "P001",
        "date": "2023-11-10",
        "amount": 14.99
      },
      {
        "product_id": "P004",
        "date": "2023-11-10",
        "amount": 17.99
      }
    ],
    "categories": [
      "cleanser",
      "moisturizer"
    ],
    "rfm": {
      "recency_days": 92,
      "frequency_90d": 0,
      "monetary_90d": 0.0
    },
    "lifetime_value": 125.80,
    "price_sensitivity": "high",
    "notes": "Inactive customer; last purchase 3 months ago. High churn risk. Missing toner, serum, SPF. Needs re-engagement."
  },
  {
    "customer_id": "C009",
    "name": "Lisa Wang",
    "email": "lisa.wang@example.com",
    "loyalty_tier": "gold",
    "churn_risk": 0.15,
    "products_owned": [
      {
        "product_id": "P003",
        "purchase_date": "2024-02-01",
        "amount": 19.99
      },
      {
        "product_id": "P004",
        "purchase_date": "2024-02-01",
        "amount": 17.99
      },
      {
        "product_id": "P009",
        "purchase_date": "2024-02-08",
        "amount": 18.99
      }
    ],
    "purchase_history": [
      {
        "product_id": "P003",
        "date": "2024-02-01",
        "amount": 19.99
      },
      {
        "product_id": "P004",
        "date": "2024-02-01",
        "amount": 17.99
      },
      {
        "product_id": "P009",
        "date": "2024-02-08",
        "amount": 18.99
      }
    ],
    "categories": [
      "serum",
      "moisturizer",
      "eye"
    ],
    "rfm": {
      "recency_days": 13,
      "frequency_90d": 3,
      "monetary_90d": 56.97
    },
    "lifetime_value": 280.50,
    "price_sensitivity": "medium",
    "notes": "Skincare enthusiast focused on anti-aging; missing cleanser, toner, SPF to complete routine."
  },
  {
    "customer_id": "C010",
    "name": "James Anderson",
    "email": "james.a@example.com",
    "loyalty_tier": "bronze",
    "churn_risk": 0.18,
    "products_owned": [
      {
        "product_id": "P010",
        "purchase_date": "2024-01-28",
        "amount": 13.99
      },
      {
        "product_id": "P002",
        "purchase_date": "2024-01-28",
        "amount": 12.99
      }
    ],
    "purchase_history": [
      {
        "product_id": "P010",
        "date": "2024-01-28",
        "amount": 13.99
      },
      {
        "product_id": "P002",
        "date": "2024-01-28",
        "amount": 12.99
      }
    ],
    "categories": [
      "cleanser",
      "toner"
    ],
    "rfm": {
      "recency_days": 24,
      "frequency_90d": 2,
      "monetary_90d": 26.98
    },
    "lifetime_value": 110.25,
    "price_sensitivity": "high",
    "notes": "Budget-conscious customer with basic routine; missing serum, moisturizer, SPF. Good candidate for value bundles."
  }
]

# data loader util

In [ ]:
"""Data loading utilities for Cross-Sell & Upsell Orchestrator"""

import json
from pathlib import Path
from typing import Dict, List, Any, Optional


def load_customer_data(customer_id: str, data_dir: str = "data") -> Optional[Dict[str, Any]]:
    """
    Load customer data by customer_id from customers.json

    Args:
        customer_id: Customer ID to load (e.g., "C001")
        data_dir: Directory containing customers.json

    Returns:
        Customer data dictionary or None if not found
    """
    customers_file = Path(data_dir) / "customers.json"

    if not customers_file.exists():
        raise FileNotFoundError(f"Customers file not found: {customers_file}")

    with open(customers_file, 'r') as f:
        customers = json.load(f)

    # Find customer by ID
    for customer in customers:
        if customer.get("customer_id") == customer_id:
            return customer

    return None


def load_all_customers(data_dir: str = "data") -> List[Dict[str, Any]]:
    """
    Load all customers from customers.json

    Args:
        data_dir: Directory containing customers.json

    Returns:
        List of all customer dictionaries
    """
    customers_file = Path(data_dir) / "customers.json"

    if not customers_file.exists():
        raise FileNotFoundError(f"Customers file not found: {customers_file}")

    with open(customers_file, 'r') as f:
        customers = json.load(f)

    return customers


def load_product_catalog(data_dir: str = "data") -> List[Dict[str, Any]]:
    """
    Load product catalog from product_catalog.json

    Args:
        data_dir: Directory containing product_catalog.json

    Returns:
        List of all product dictionaries
    """
    catalog_file = Path(data_dir) / "product_catalog.json"

    if not catalog_file.exists():
        raise FileNotFoundError(f"Product catalog file not found: {catalog_file}")

    with open(catalog_file, 'r') as f:
        catalog = json.load(f)

    return catalog


def build_product_lookup(catalog: List[Dict[str, Any]]) -> Dict[str, Dict[str, Any]]:
    """
    Build a fast lookup dictionary: product_id -> product dict

    Args:
        catalog: List of product dictionaries

    Returns:
        Dictionary mapping product_id to product data
    """
    lookup = {}
    for product in catalog:
        product_id = product.get("product_id")
        if product_id:
            lookup[product_id] = product
    return lookup


def get_essential_categories() -> List[str]:
    """
    Get list of essential skincare routine categories (Tier 1)

    Returns:
        List of essential category names
    """
    return ["cleanser", "toner", "serum", "moisturizer", "spf"]


def get_all_categories(catalog: List[Dict[str, Any]]) -> List[str]:
    """
    Get all unique categories from product catalog

    Args:
        catalog: List of product dictionaries

    Returns:
        Sorted list of unique category names
    """
    categories = set()
    for product in catalog:
        category = product.get("category")
        if category:
            categories.add(category)
    return sorted(list(categories))



# test data utils

In [ ]:
"""Tests for data loading utilities"""

import pytest
from pathlib import Path
import sys

# Add src to path
sys.path.insert(0, str(Path(__file__).parent.parent / "src"))

from cross_sell_upsell.data_utils import (
    load_customer_data,
    load_all_customers,
    load_product_catalog,
    build_product_lookup,
    get_essential_categories,
    get_all_categories
)


def test_load_customer_data():
    """Test loading a specific customer"""
    customer = load_customer_data("C001")

    assert customer is not None
    assert customer["customer_id"] == "C001"
    assert customer["name"] == "Sarah Lee"
    assert "products_owned" in customer
    assert "categories" in customer
    assert "rfm" in customer
    assert "lifetime_value" in customer


def test_load_customer_data_not_found():
    """Test loading non-existent customer"""
    customer = load_customer_data("C999")
    assert customer is None


def test_load_all_customers():
    """Test loading all customers"""
    customers = load_all_customers()

    assert len(customers) > 0
    assert all("customer_id" in c for c in customers)
    assert all("name" in c for c in customers)


def test_load_product_catalog():
    """Test loading product catalog"""
    catalog = load_product_catalog()

    assert len(catalog) > 0
    assert all("product_id" in p for p in catalog)
    assert all("category" in p for p in catalog)
    assert all("price" in p for p in catalog)
    assert all("recommended_cross_sells" in p for p in catalog)


def test_build_product_lookup():
    """Test building product lookup dictionary"""
    catalog = load_product_catalog()
    lookup = build_product_lookup(catalog)

    assert len(lookup) == len(catalog)
    assert "P001" in lookup
    assert lookup["P001"]["name"] == "Gentle Foaming Cleanser"
    assert lookup["P001"]["category"] == "cleanser"


def test_get_essential_categories():
    """Test getting essential categories"""
    essential = get_essential_categories()

    assert "cleanser" in essential
    assert "toner" in essential
    assert "serum" in essential
    assert "moisturizer" in essential
    assert "spf" in essential
    assert len(essential) == 5


def test_get_all_categories():
    """Test getting all categories from catalog"""
    catalog = load_product_catalog()
    categories = get_all_categories(catalog)

    assert "cleanser" in categories
    assert "toner" in categories
    assert "serum" in categories
    assert "moisturizer" in categories
    assert "spf" in categories
    assert len(categories) >= 5  # At least essential categories



# test results

In [ ]:
(.venv) micahshull@Micahs-iMac LG_Cursor_029_CrossSell_Upsell_Orchestrator % cd /Users/micahshull/Documents/AI_LangGraph/LG_Cursor_029_CrossSell_Upsell_Orchestrator && python3 -m pytest tests/test_data_utils.py -v
=================================================================== test session starts ===================================================================
platform darwin -- Python 3.13.7, pytest-9.0.1, pluggy-1.6.0 -- /Users/micahshull/Documents/AI_LangGraph/LG_Cursor_029_CrossSell_Upsell_Orchestrator/.venv/bin/python3
cachedir: .pytest_cache
rootdir: /Users/micahshull/Documents/AI_LangGraph/LG_Cursor_029_CrossSell_Upsell_Orchestrator
plugins: anyio-4.11.0, asyncio-1.3.0, langsmith-0.4.44, cov-7.0.0
asyncio: mode=Mode.STRICT, debug=False, asyncio_default_fixture_loop_scope=None, asyncio_default_test_loop_scope=function
collected 7 items

tests/test_data_utils.py::test_load_customer_data PASSED                                                                                            [ 14%]
tests/test_data_utils.py::test_load_customer_data_not_found PASSED                                                                                  [ 28%]
tests/test_data_utils.py::test_load_all_customers PASSED                                                                                            [ 42%]
tests/test_data_utils.py::test_load_product_catalog PASSED                                                                                          [ 57%]
tests/test_data_utils.py::test_build_product_lookup PASSED                                                                                          [ 71%]
tests/test_data_utils.py::test_get_essential_categories PASSED                                                                                      [ 85%]
tests/test_data_utils.py::test_get_all_categories PASSED                                                                                            [100%]

==================================================================== 7 passed in 0.02s ====================================================================


